In [2]:
#import libs
import os, glob
import pandas as pd
import plotly.offline as pyo
import plotly.graph_objs as go
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
from datetime import date, timedelta
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import warnings
warnings.simplefilter("ignore")

In [3]:
DATA_PATH = '../data/processed/Alarms_data_processed.parquet'

In [4]:
df = pd.read_parquet(DATA_PATH)
df.head(1)

,Severity,Name,Alarm Source,Occurred On (NT),Cleared On (NT),Type,Alarm Duration,Alarm_Duration_Hour
40,Major,IKE Negotiation Failure,LSINS38284,2021-10-13 14:38:00,2021-10-14 10:38:00,Trunk system,19 hours 59 minutes 40 seconds,20.0


In [5]:
# MBA one Site
snap = df[df['Alarm Source']=='LCAIN12215'].copy()
snap['q'] = 1
snap = snap[['Name','Alarm Source','Occurred On (NT)','q']]
basket = snap.groupby(['Alarm Source','Occurred On (NT)','Name'])['q'].sum().unstack().reset_index().fillna(0).set_index('Occurred On (NT)')
basket.drop(columns=['Alarm Source'],inplace=True)

def encode(x):
    if x <=0:
        return 0
    if x >=1:
        return 1

basket_encode = basket.applymap(encode)

basket_plus = basket_encode[(basket_encode>0).sum(axis=1)>=2]

In [6]:
basket_plus.head(1)

Name,Cell RX Channel Interference Noise Power Unbalanced,External Clock Reference Problem,FAN Fault Alarm,IKE Negotiation Failure,IP Clock Link Failure,License on Trial,SCTP Link Fault,User Plane Fault,User Plane Path Fault,X2 Interface Fault
Occurred On (NT),,,,,,,,,,
2021-10-13 14:41:00,0,0,0,1,0,0,1,0,0,0


In [7]:
freq_pattern = apriori(basket_plus,min_support=0.03,use_colnames=True).sort_values('support',ascending=False).reset_index(drop=True)
freq_pattern['len'] = freq_pattern['itemsets'].apply(lambda x: len(x))
freq_pattern[freq_pattern['len']>1]

,support,itemsets,len
2,0.666667,"(SCTP Link Fault, User Plane Path Fault)",2
4,0.200000,"(SCTP Link Fault, IKE Negotiation Failure)",2
6,0.133333,"(SCTP Link Fault, License on Trial)",2
8,0.066667,"(X2 Interface Fault, License on Trial)",2
9,0.066667,"(X2 Interface Fault, SCTP Link Fault)",2
10,0.066667,"(X2 Interface Fault, SCTP Link Fault, License ...",3


In [8]:
AR = association_rules(freq_pattern,metric='lift',min_threshold=1).sort_values('lift',ascending=False).reset_index(drop=True)
AR[AR['conviction']!=np.inf]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(License on Trial),(X2 Interface Fault),0.133333,0.066667,0.066667,0.500000,7.5,0.057778,1.866667
3,"(SCTP Link Fault, License on Trial)",(X2 Interface Fault),0.133333,0.066667,0.066667,0.500000,7.5,0.057778,1.866667
5,(License on Trial),"(X2 Interface Fault, SCTP Link Fault)",0.133333,0.066667,0.066667,0.500000,7.5,0.057778,1.866667
6,(SCTP Link Fault),(User Plane Path Fault),1.000000,0.666667,0.666667,0.666667,1.0,0.000000,1.000000
8,(SCTP Link Fault),(IKE Negotiation Failure),1.000000,0.200000,0.200000,0.200000,1.0,0.000000,1.000000
10,(SCTP Link Fault),(License on Trial),1.000000,0.133333,0.133333,0.133333,1.0,0.000000,1.000000
13,(SCTP Link Fault),(X2 Interface Fault),1.000000,0.066667,0.066667,0.066667,1.0,0.000000,1.000000
15,(SCTP Link Fault),"(X2 Interface Fault, License on Trial)",1.000000,0.066667,0.066667,0.066667,1.0,0.000000,1.000000


In [9]:
# MBA on All sites
def encode(x):
        if x <=0:
            return 0
        if x >=1:
            return 1
        
for i in df['Alarm Source'].unique()[11:30]:
    snap = df[df['Alarm Source']==i].copy()
    snap['q'] = 1
    snap = snap[['Name','Alarm Source','Occurred On (NT)','q']]
    basket = snap.groupby(['Alarm Source','Occurred On (NT)','Name'])['q'].sum().unstack().reset_index().fillna(0).set_index('Occurred On (NT)')
    basket.drop(columns=['Alarm Source'],inplace=True)
    basket_encode = basket.applymap(encode)
    basket_plus = basket_encode[(basket_encode>0).sum(axis=1)>=2]
    freq_pattern = apriori(basket_plus,min_support=0.03,use_colnames=True).sort_values('support',ascending=False).reset_index(drop=True)
    freq_pattern['len'] = freq_pattern['itemsets'].apply(lambda x: len(x))
    freq_pattern[freq_pattern['len']>1].to_csv(r"..\data\interim\MBA\{}.csv".format(i))